# TX00DQ05-3001 Exercises 3

In [4]:
import numpy as np
import numpy.linalg as LA
import random

## Exercise 1: Sample behaviour of an MDP

Let's take (again) a look at Sutton & Barto example 4.1 gridworld. On each iteration start at every (non-terminating) state and sample actions in succeeding states by selecting them from uniform distribution (each action - up, down, left, right - is equally probable). Run the episode until terminal state is encountered. Collect statistics to calculate average number of steps needed before completion for each start state. Should this number match with something you have seen earlier in the exercises?

In [67]:

def calculateValueForCurrentState(value,stepcost):
    #print("(",stepcost,"+",value,")*0.25 = ",((stepcost+(value))*0.25))
    return ((stepcost+(value))*0.25)


def checkWithBoundary(max,r,c,oR,oC):
    #print("-nstate: ",r,",",c)
    #print(V)
    if 0 > r or max <= r or 0 > c or max <= c :
        #print(oR,",",oC," value:",V[oR,oC])
        return (oR,oC)
    else:
        #print(r,",",c," value:",V[r,c])
        return (r,c)

    
    
def Actions(val):
    if val == 0:
        return [1,0]
    elif val == 1:
        return [-1,0]
    elif val == 2:
        return [0,1]
    elif val == 3:
        return [0,-1]
    

def randomAction():
    return Actions(random.randint(0,3))

def makeAction(maxsize,r,c):
    a = randomAction()
    
    
    return checkWithBoundary(maxsize,r+a[0],c+a[1],r,c)

def checkIfInTerminatingPoint(pos,term):
    for t in term:
        if pos == t:
            return True
    return False
    
class Agent(object):
    steps = 0
    def __init__(self, startPos):
        self.startPos = startPos
        self.pos = startPos
    
def MC_pathlengths(maxiters,gridworld,terminating):
    
    lengths= gridworld
    agents =[]
    
    for x in range(lengths.shape[0]):
        for y in range(lengths.shape[1]):
            if not checkIfInTerminatingPoint((x,y),terminating):
                agents.append( Agent((x,y)) )
    print(len(agents))
    for i in range(1, maxiters):

        goals = []
        calc = 0
        
        while (len(agents)>0):
            for agent in agents:
                agent.pos = makeAction(lengths.shape[0],agent.pos[0],agent.pos[1])
                agent.steps += 1
                if (checkIfInTerminatingPoint(agent.pos,terminating)):
                    agents.remove(agent)
                    goals.append(agent)
                    break
                    
            
        for agent in goals:
            mean = ((i-1)/i) * lengths[agent.startPos] + agent.steps/i
            #print(((i-1)/i),"*",lengths[agent.startPos],"+",agent.steps/i,"=",mean)
            lengths[agent.startPos] = mean
            agent.steps = 0
            agent.pos = agent.startPos
            
        agents = goals
    return lengths       
    
    
def CreateGridWorld(gridsize,terminating = None,numberOfTerminating = None):
    

    if(terminating == None):
        if(numberOfTerminating == 0 or numberOfTerminating == None):
            numberOfTerminating = 1
        terminating = []
        for n in range(numberOfTerminating):
            terminating.append((random.randint(0,gridsize[0]-1), \
                        random.randint(0,gridsize[1]-1)))
        
        
    gridworld = np.zeros((gridsize[0], gridsize[1]))
    
    print(terminating)
    print(gridworld)
    return gridworld, terminating


gridworld,terminating = CreateGridWorld((4,4),[(0,0), (3,3)],)
with np.printoptions(precision=2):
    print(MC_pathlengths(1000,gridworld,terminating))

[(0, 0), (3, 3)]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
14
[[0.   0.11 0.29 0.3 ]
 [0.13 0.18 0.17 0.21]
 [0.28 0.27 0.13 0.14]
 [0.25 0.24 0.15 0.  ]]


## Exercise 2: Monte Carlo state value function estimation. 

Calculate state-value function V for the gridworld of Sutton & Barto example 4.1 using first-visit or every-visit Monte Carlo policy evaluation (see for example page 92 of Sutton & Barto). Policy to be evaluated is the same as before; each action (up, down, left, right) is equally probable.  Action that would result in leaving the grid (for example moving up in top row) will leave state unchanged (but action has been taken). Gamma (discount factor) is assumed to be = 1, ie. no discounting.

Try out both exploring starts (see Sutton & Barto, p. 96) and fixed start points. Any difference?

Take a look at the value function you get when you run the algorithm multiple times (with fixed # of iterations). Any observations?

In [111]:
def MC_randomwalk(maxiters,gridworld,terminating):
    
    lengths= gridworld
    agents =[]
    
    for x in range(lengths.shape[0]):
        for y in range(lengths.shape[1]):
            if not checkIfInTerminatingPoint((x,y),terminating):
                agents.append( Agent((x,y)) )
    print(len(agents))
    for i in range(1, maxiters):

        goals = []
        calc = 0
        
        
        for agent in agents:
            steps = []
            steps.append(agent.pos)
            while(True):
                agent.pos = makeAction(lengths.shape[0],agent.pos[0],agent.pos[1])
                
                agent.steps += 1
                if (checkIfInTerminatingPoint(agent.pos,terminating)):
                    goals.append(agent)
                    break
                steps.append(agent.pos)    
            num =1
            agent.pos = agent.startPos
            steps.reverse()
            for step in steps:
                
                mean= (lengths[step]-1/num)
                #print(lengths[step],"- 1 /",num," = ",mean)
                
                lengths[step]=mean
                
        
    
    return lengths/100

gridworld,terminating = CreateGridWorld((4,4),[(0,0), (3,3)],)
with np.printoptions(precision=2):
    print(MC_randomwalk(100,gridworld,terminating))

[(0, 0), (3, 3)]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
14
[[  0.   -13.97 -18.04 -21.17]
 [-14.7  -17.93 -18.43 -20.03]
 [-21.08 -19.73 -17.05 -13.61]
 [-23.94 -19.62 -13.19   0.  ]]


## Exercise 3*: Monte Carlo action value function estimation

Use the same idea as in exercise 2 to estimate q function.

*) - not mandatory

## Exercise 4*: Monte Carlo control

Compute the optimal policy for the 4x4 gridworld example. Start with random policy. Consider the epsilon adjustment schedule - can it in practise be 1/k, or is something more conservative better? Can you think of any other tricks to manage the noisiness of MC?

*) - not mandatory